In [1]:
from splinter import Browser
import pandas as pd
import numpy as np
import time
from collections import defaultdict
import boto3
from boto3.dynamodb.conditions import Key, Attr
import json
from decimal import Decimal
import requests
import re
import uuid

In [2]:
start = "10/20/19"
end = "10/27/19"
start_unix = int(time.mktime(pd.datetime.strptime(start, "%m/%d/%y" ).timetuple())*1000)
end_unix = int(time.mktime(pd.datetime.strptime(end, "%m/%d/%y" ).timetuple())*1000)
period_string = pd.datetime.strptime(start, "%m/%d/%y" ).strftime("%Y%m%d") + pd.datetime.strptime(end, "%m/%d/%y" ).strftime("%Y%m%d")

In [5]:
session = requests.Session()
dynamodb = boto3.resource('dynamodb', region_name='us-east-1')
table = dynamodb.Table('shift')


In [6]:
type(dynamodb.tables)

boto3.resources.collection.dynamodb.tablesCollectionManager

In [7]:
table.batch_writer()

In [8]:
fe = Key("scheduled_start").gt(start_unix) & Key("scheduled_start").lt(end_unix) & Key("shift_status").gt(0)

In [9]:
resp = table.scan( FilterExpression = fe)['Items']

In [10]:
shifts = pd.DataFrame(resp)

In [11]:
def calc_hours(x, break_thresh = 6):
    logged_hours = Decimal((x['end']-x['start'])/(1000*60*60))
    if (logged_hours > break_thresh):
        return logged_hours - Decimal(0.5)
    else:
        return logged_hours

In [12]:
shifts['adj_hours'] = shifts.apply(lambda x: calc_hours(x),axis = 1)

In [13]:
shifts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 18 columns):
app_count          43 non-null object
carrier            43 non-null object
carrier_id         43 non-null object
created_dt         43 non-null object
description        1 non-null object
driver             43 non-null object
end                43 non-null object
id                 43 non-null object
modified_dt        43 non-null object
scheduled_start    43 non-null object
shift_status       43 non-null object
start              43 non-null object
start_address      43 non-null object
start_city         43 non-null object
start_state        43 non-null object
start_zip          43 non-null object
user_id            43 non-null object
adj_hours          43 non-null object
dtypes: object(18)
memory usage: 6.1+ KB


In [14]:
%load_ext autoreload
%autoreload 2

In [15]:
payroll_rates = defaultdict(int)
client_rates = defaultdict(int)
for name in shifts['driver'].unique():
    if name == "William Baldeo":
        payroll_rates[name]=30
    elif name == "Maurice  Gaskins ":
        payroll_rates[name]=20
    elif name == "Robert Velicky":
        payroll_rates[name]=27
    elif name == "Willie Thomas":
        payroll_rates[name]=26
    else:
        payroll_rates[name]=25
client_rates['Bettaway Transportation Logistics'] = 34
client_rates['Universal Supply Company'] = 44
client_rates['Lotus Scoop Ice Cream'] = 30

In [16]:
shifts['Payroll Rate'] = shifts.apply(lambda x: payroll_rates[x['driver']],axis = 1)
shifts['Client Rate'] = shifts.apply(lambda x: client_rates[x['carrier']],axis = 1)
shifts['Payroll Amount'] = shifts['adj_hours'] * shifts['Payroll Rate']
shifts['Billed Amount'] = shifts['adj_hours'] * shifts['Client Rate']
shifts['break_hours']=shifts['adj_hours'].apply(lambda x: Decimal(0) if x < 5.5 else Decimal(0.5))
shifts['start_time']= shifts['start'].apply(lambda x: pd.datetime.fromtimestamp(x/1000).strftime("%H:%M"))
shifts['end_time']= shifts['end'].apply(lambda x: pd.datetime.fromtimestamp(x/1000).strftime("%H:%M"))
shifts['date']= shifts['scheduled_start'].apply(
    lambda x: pd.datetime.fromtimestamp(x/1000).strftime("%m/%d/%y"))

In [17]:
shifts.sort_values(by=['driver','date'],inplace=True)

In [18]:
print("Gross Revenue: ${:.02f}".format(shifts['Billed Amount'].sum()))
print("Driver Payroll: ${:.02f}".format(shifts['Payroll Amount'].sum()))
print("Gross Profit: ${:.02f}".format(shifts['Billed Amount'].sum()-shifts['Payroll Amount'].sum()))

Gross Revenue: $16830.00
Driver Payroll: $12419.00
Gross Profit: $4411.00


In [19]:
shifts['description']=shifts['description'].apply(lambda x: "Class A" if pd.isna(x) else x)

In [20]:
shifts['Payroll Amount'] = shifts.apply(lambda x: x['Payroll Amount'] + Decimal(12.5) if x['driver']=='Roberto Rodriguez' else x['Payroll Amount'] ,axis = 1)

In [21]:
print("Hours Billed: {:.01f}".format(shifts['adj_hours'].sum()))
print("Gross Revenue: ${:.02f}".format(shifts['Billed Amount'].sum()))
print("Driver Payroll: ${:.02f}".format(shifts['Payroll Amount'].sum()))
print("Gross Profit: ${:.02f}".format(shifts['Billed Amount'].sum()-shifts['Payroll Amount'].sum()))

Hours Billed: 495.0
Gross Revenue: $16830.00
Driver Payroll: $12481.50
Gross Profit: $4348.50


In [22]:
export_cols = ['date','carrier','description','driver','shift_status','start_address','start_city','start_state',
               'adj_hours','Payroll Rate','Client Rate','Payroll Amount','Billed Amount','break_hours',
              'start_time','end_time']
shifts[export_cols].to_csv("weekly_billing_{}.csv".format(period_string[8:]))

pd.datetime.strptime(shifts['Log date'].iloc[0],"%m/%d/%Y") > pd.datetime.strptime("08/03/2019","%m/%d/%Y")

In [23]:
session = requests.Session()
dynamodb = boto3.resource('dynamodb', region_name='us-east-2')
table = dynamodb.Table('invoice')


In [24]:
def create_invoice(table, carrier,carrier_id,des,start_date,end_date, period_string):
    invoice_id = str(uuid.uuid4())
    qb_id = int(time.time()*1000)    
    resp = table.update_item(Key = {'id':invoice_id},
                ExpressionAttributeNames={
                    "#qb_id" : "qb_id",
                    "#created_dt" : "created_dt",
                    "#des" : "description",
                    "#invoice_date":"invoice_date",
                    "#carrier": "carrier",
                    "#carrier_id": "carrier_id",
                    "#period":"period",
                },
                ExpressionAttributeValues = {
                    ":qb_id":qb_id,
                    ":carrier" : carrier,
                    ":carrier_id" : carrier_id,
                    ":des" : ' '.join([des,start_date,end_date]),
                    ":period": period_string,
                },
                UpdateExpression = """SET
                    #qb_id = :qb_id,
                    #created_dt = :qb_id,
                    #invoice_date = :qb_id,
                    #des = :des,
                    #carrier = :carrier,
                    #carrier_id = :carrier_id,
                    #period = :period
                """
                )
    return [invoice_id,qb_id]

In [25]:
def invoice_item(x,table, invoice_id, qb_id,period_string):
    item_id = str(uuid.uuid4())
    created_dt = int(time.time()*1000)
    adj_time = x["adj_hours"].quantize(Decimal("0.01"))
    carrier_rate = Decimal.from_float(x["Client Rate"]).quantize(Decimal("0.01"))
    resp = table.update_item(Key = {'id':item_id},
                ExpressionAttributeNames={
                    "#adj_time" : "adj_time",
                    "#break_hours" : "break_hours",
                    "#carrier_rate" : "carrier_rate",
                    "#carrier_total" : "carrier_total",
                    "#created_dt" : "created_dt",
                    "#date" : "date",
                    "#des" : "description",
                    "#driver" : "driver",
                    "#driver_rate" : "driver_rate",
                    "#driver_total" : "driver_total",
                    "#start_time" : "start_time",
                    "#end_time" : "end_time",
                    "#invoice_id" : "invoice_id",
                    "#qb_id" : "qb_id",
                    "#period":"period",
                },
                ExpressionAttributeValues= {
                    ":adj_time" : adj_time,
                    ":break_hours" : x['break_hours'],
                    ":carrier_rate" : carrier_rate,
                    ":carrier_total" : adj_time * carrier_rate,
                    ":created_dt" : created_dt,
                    ":date" : x['date'],
                    ":description" : x['description'],
                    ":driver" : x['driver'],
                    ":driver_rate" : Decimal.from_float(x["Payroll Rate"]).quantize(Decimal("0.01")),
                    ":driver_total" : x["Payroll Amount"].quantize(Decimal("0.01")),
                    ":start_time" : x["start_time"],
                    ":end_time" : x["end_time"],
                    ":invoice_id" : invoice_id,
                    ":qb_id" : qb_id,
                    ":period": period_string,
                },
                UpdateExpression = """SET
                    #adj_time = :adj_time,
                    #break_hours = :break_hours,
                    #carrier_rate = :carrier_rate,
                    #carrier_total = :carrier_total,
                    #created_dt = :created_dt,
                    #date = :date,
                    #des = :description,
                    #driver = :driver,
                    #driver_rate = :driver_rate,
                    #driver_total = :driver_total,
                    #start_time = :start_time,
                    #end_time = :end_time,
                    #invoice_id = :invoice_id,
                    #qb_id = :qb_id,
                    #period = :period
                """
                    )
    return True

In [26]:
dynamodb = boto3.resource('dynamodb', region_name='us-east-1')
table = dynamodb.Table('invoice')
carrier = "Bettaway Transportation Logistics"
carrier_id = "27ea2970-bac2-11e9-9172-918bcb645def"
start_date = "09/22/2019"
end_date = "09/29/2019"
des = "Class A Work"
invoice_id,qb_id = create_invoice(table, carrier,carrier_id,des,start_date,end_date,period_string)

In [27]:
print(invoice_id)
print(qb_id)
table = dynamodb.Table('invoice_item')
filter_shifts = shifts[shifts['carrier']==carrier]
filter_shifts.apply(lambda x: invoice_item(x,table,invoice_id,qb_id, period_string),axis = 1)

f87d0dc5-6b21-4f0c-a12e-683213565ceb
1572458240263


14    True
18    True
35    True
7     True
20    True
3     True
34    True
0     True
12    True
19    True
4     True
28    True
6     True
1     True
36    True
13    True
25    True
31    True
9     True
5     True
23    True
42    True
39    True
11    True
10    True
2     True
38    True
15    True
33    True
32    True
40    True
24    True
30    True
22    True
41    True
17    True
27    True
26    True
16    True
8     True
21    True
37    True
29    True
dtype: bool

In [81]:
dynamodb = boto3.resource('dynamodb', region_name='us-east-1')
table = dynamodb.Table('invoice')
carrier = "Universal Supply Company"
carrier_id = "d8201570-e39e-11e9-9d02-fde362fdfd63"
start_date = "09/22/2019"
end_date = "09/29/2019"
des = "Class A Work"
invoice_id,qb_id = create_invoice(table, carrier,carrier_id,des,start_date,end_date,period_string)

In [82]:
print(invoice_id)
print(qb_id)
table = dynamodb.Table('invoice_item')
filter_shifts = shifts[shifts['carrier']==carrier]
filter_shifts.apply(lambda x: invoice_item(x,table,invoice_id,qb_id, period_string),axis = 1)

6011dfb5-ef64-47db-bf72-b5f55b34dedf
1570717966536


14    True
dtype: bool

In [18]:
dynamodb = boto3.resource('dynamodb', region_name='us-east-1')
table = dynamodb.Table('invoice')
carrier = "Lotus Scoop Ice Cream"
carrier_id = "2e990320-c9c9-11e9-b35f-5356449f14da"
start_date = "08/18/2019"
end_date = "08/25/2019"
des = "Non-commercial van"
invoice_id,qb_id = create_invoice(table, carrier,carrier_id,des,start_date,end_date,period_string)


In [19]:
print(invoice_id)
print(qb_id)
table = dynamodb.Table('invoice_item')
filter_shifts = shifts[shifts['carrier']==carrier]
filter_shifts.apply(lambda x: invoice_item(x,table,invoice_id,qb_id, period_string),axis = 1)

72cd878c-cf36-4740-a1b5-8896035fe7e7
1568985162326


4    True
dtype: bool

In [61]:
shifts.to_csv("weekly_billing_{}.csv".format(period_string[8:]))

In [28]:
shifts[['driver','Payroll Amount','adj_hours']].groupby(by = "driver").apply(sum).iloc[:,1:].to_csv(
    "driver_payroll_{}.csv".format(period_string[8:]))

In [29]:
shifts[['carrier','Billed Amount','adj_hours']].groupby(by = "carrier").apply(sum).iloc[:,1:].to_csv(
    "carrier_totals_{}.csv".format(period_string[8:]))

In [30]:
export_cols = ['date','carrier','description','driver','shift_status','start_address','start_city','start_state',
               'adj_hours','Payroll Rate','Client Rate','Payroll Amount','Billed Amount','break_hours',
              'start_time','end_time']
shifts[export_cols].to_csv("weekly_billing_{}.csv".format(period_string[8:]))

In [31]:
shifts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43 entries, 14 to 29
Data columns (total 26 columns):
app_count          43 non-null object
carrier            43 non-null object
carrier_id         43 non-null object
created_dt         43 non-null object
description        43 non-null object
driver             43 non-null object
end                43 non-null object
id                 43 non-null object
modified_dt        43 non-null object
scheduled_start    43 non-null object
shift_status       43 non-null object
start              43 non-null object
start_address      43 non-null object
start_city         43 non-null object
start_state        43 non-null object
start_zip          43 non-null object
user_id            43 non-null object
adj_hours          43 non-null object
Payroll Rate       43 non-null int64
Client Rate        43 non-null int64
Payroll Amount     43 non-null object
Billed Amount      43 non-null object
break_hours        43 non-null object
start_time         43 no